# Demo of the trait exception rewriting

Traitlets results in very long stacktraces.  Look at the difference between these two stack traces.

Watch this space,  These stacktraces still aren't great, but they are much better

In [1]:
import sys
from traitlets import Unicode, Any, observe, HasTraits, Dict
import six

In [2]:
class DumbBase(HasTraits):
    first_trait = Any()
    second_trait = Any()
    third_trait = Any()

    def __init__(self, val):
        super().__init__()
        self.first_trait = val
        
    def compute_first(self, val):
        return val
    
    @observe('first_trait')
    def handle_first(self, change):
        self.second_trait = self.compute_first(self.first_trait)

    def compute_second(self, val):
        return val
    
    @observe('second_trait')
    def handle_second(self, change):
        self.third_trait = self.compute_second(self.second_trait)

class ErrorSecond(DumbBase):
    def compute_second(self, val):
        1/0
        
ErrorSecond(5)

ZeroDivisionError: division by zero

In [4]:
def exception_protect(func):
    def wrapped(self, *args, **kwargs):
        try:
            func(self, *args, **kwargs)
        except Exception as e:
            if self.exception is None:
                self.exception = sys.exc_info()
            raise
    return wrapped

class SmartBase(HasTraits):
    first_trait = Any()
    second_trait = Any()
    third_trait = Any()

    def __init__(self, val):
        super().__init__()
        self.exception = None
        try:
            self.first_trait = val
        except Exception as e:
            # print("e", sys.exc_info())
            print("self.exception", self.exception)
            exc, exc1, tb = self.exception
            six.reraise(exc, exc1, tb)
        
    def compute_first(self, val):
        return val
    
    @observe('first_trait')
    @exception_protect
    def handle_first(self, change):
        self.second_trait = self.compute_first(self.first_trait)

    def compute_second(self, val):
        return val
    
    @observe('second_trait')
    @exception_protect
    def handle_second(self, change):
        try:
            self.third_trait = self.compute_second(self.second_trait)
        except Exception as e:
            self.exception = sys.exc_info()
            raise

class ErrorSecondSmart(SmartBase):
    def compute_second(self, val):
        1/0
ab = ErrorSecondSmart(5)
ab

self.exception (<class 'ZeroDivisionError'>, ZeroDivisionError('division by zero'), <traceback object at 0x106fb5880>)


ZeroDivisionError: division by zero